In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)

2.20.0


In [2]:
True_News = pd.read_csv("FakeNews Dataset/True.csv")
True_News['target'] = 1
Fake_News = pd.read_csv("FakeNews Dataset/Fake.csv")
Fake_News['target'] = 0

In [3]:
True_News.drop('date', axis =1, inplace =True)

In [4]:
Fake_News.drop('date', axis =1, inplace = True)

In [5]:
df = pd.concat([True_News, Fake_News], ignore_index=True )

In [6]:
df

,title,text,subject,target
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,1
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,1
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,1
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,1
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,1
...,...,...,...,...
44893,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,0
44894,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,0
44895,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,0
44896,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,0


In [7]:
df = df.sample(frac = 1.0, ignore_index=True)
df

,title,text,subject,target
0,Britain and France to work to enforce Iran nuc...,LONDON (Reuters) - Britain and France are firm...,worldnews,1
1,State Department Confirms Hillary Clinton Did...,"Another day, another Hillary Clinton scandal h...",News,0
2,Billionaire GOP Donors REFUSE To Crown King T...,"In 2012, when we were still smarting from the ...",News,0
3,"WOW! CHICAGO REPORTEDLY FINDS At Least 14,000 ...",The head of the Chicago Republican Party is cl...,left-news,0
4,EU's new eastern leaders stick to guns on refu...,BRUSSELS (Reuters) - The new prime ministers o...,worldnews,1
...,...,...,...,...
44893,TRUMP IS RIGHT AGAIN! While Pundits and Politi...,This is an odd report from Press TV because a ...,politics,0
44894,London police say have stood down after Oxford...,LONDON (Reuters) - London s police force said ...,worldnews,1
44895,"Angry over U.S. healthcare fail, Trump voters ...","NEW PORT RICHEY, Fla. (Reuters) - The day aft...",politicsNews,1
44896,IT BEGINS: Man Dressed As Woman Arrested In Wo...,We told you so a man was having a blast peepin...,left-news,0


In [8]:
df.shape

(44898, 4)

In [9]:
X = df.drop('target', axis = 1) 
y = df.target

In [10]:
X.shape ,y.shape

((44898, 3), (44898,))

In [11]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [20]:
voc_size = 5000

In [13]:
news = X.copy()

In [14]:
news['title'][0]

"Britain and France to work to enforce Iran nuclear deal: UK PM May's office"

In [15]:
!pip install nltk


In [16]:
import nltk
import re
from nltk.corpus import stopwords

In [17]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Gaurav\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [18]:
import tensorflow as tf

print("TensorFlow version:", tf.__version__)
print("GPUs available:", tf.config.list_physical_devices('GPU'))


TensorFlow version: 2.20.0
GPUs available: []


In [19]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(news)):
    review = re.sub('[^a-zA-Z]', ' ', news['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [22]:
one_hot_repr = [one_hot(words, voc_size) for words in corpus]
one_hot_repr

[[4587, 59, 4283, 3660, 2300, 4446, 3309, 1245, 65, 2358, 3448],
 [3251, 2565, 1809, 4898, 4654, 3742, 3414, 4163, 2669],
 [3453, 1334, 4417, 3743, 2521, 4335, 2863, 3211, 4377, 1025],
 [2140, 26, 3923, 744, 116, 2527, 3430, 1221],
 [2430, 3524, 3643, 4771, 2096, 2734, 578],
 [1633, 1535, 3742, 999, 2863, 3628, 3984],
 [2863, 1535, 4900, 2049, 4357, 2987, 2933, 3129],
 [3535, 1119, 2796, 4210, 2863, 2697],
 [3653, 4898, 4668, 1087, 2841, 4159, 1212],
 [3116,
  1481,
  1993,
  1115,
  3475,
  592,
  2862,
  2811,
  2205,
  3524,
  4645,
  2495,
  3478,
  3465,
  4270,
  3814],
 [2863, 2705, 3653, 1685, 3251, 3210, 3264, 3218, 976, 578, 3492, 1481],
 [4037, 4583, 2863, 1072, 2399, 3247, 1491, 1974],
 [3211, 4432, 2714, 2531, 4182, 1786, 4490, 2216],
 [1115, 69, 1384, 3411, 4579, 2863],
 [3768, 3240, 3524, 3337, 642, 2863, 4859, 4084, 875, 3349],
 [342, 4898, 3494, 2863, 4061, 1960, 1619, 22, 592, 3197],
 [2025, 270, 365, 703, 910, 3643, 3632],
 [4577, 538, 22, 614, 1915, 4549, 4419, 1210

In [23]:
sent_length = 20

In [26]:
embedded_docs = pad_sequences(one_hot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...   65 2358 3448]
 [   0    0    0 ... 3414 4163 2669]
 [   0    0    0 ... 3211 4377 1025]
 ...
 [   0    0    0 ... 3430 2302 1096]
 [   0    0    0 ... 4310 1181 4900]
 [   0    0    0 ... 1528 2933 3741]]


In [27]:
embedded_docs[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0, 4587,   59,
       4283, 3660, 2300, 4446, 3309, 1245,   65, 2358, 3448], dtype=int32)

In [30]:
from tensorflow.keras.layers import Dropout

embedding_vector_features = 40 
model = Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy' , optimizer = 'adam' , metrics = ['accuracy'])
print(model.summary())

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm (LSTM)                          │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ ?                           │     0 (unbuilt) │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [31]:
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [32]:
X_final.shape,y_final.shape

((44898, 20), (44898,))

In [33]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [34]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 9s 13ms/step - accuracy: 0.8983 - loss: 0.2412 - val_accuracy: 0.9370 - val_loss: 0.1585
Epoch 2/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9486 - loss: 0.1322 - val_accuracy: 0.9347 - val_loss: 0.1628
Epoch 3/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9605 - loss: 0.1034 - val_accuracy: 0.9418 - val_loss: 0.1444
Epoch 4/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 13ms/step - accuracy: 0.9683 - loss: 0.0842 - val_accuracy: 0.9412 - val_loss: 0.1601
Epoch 5/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9740 - loss: 0.0690 - val_accuracy: 0.9425 - val_loss: 0.1757
Epoch 6/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9788 - loss: 0.0566 - val_accuracy: 0.9411 - val_loss: 0.1944
Epoch 7/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9811 - loss: 0.0498 - val_accuracy: 0.9389 - val_loss: 0.2021
Epoch 8/10
471/471 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.9843 - loss: 0.0430 - val_accu

In [40]:
y_pred=model.predict(X_test)
y_pred = (y_pred > 0.5)

464/464 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step


In [41]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,y_pred)

array([[7226,  472],
       [ 406, 6713]])

In [42]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.940743740298306